# 1. Import Dependencies

In [1]:
# Import Gym Stuff
import gym
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete

# Import Helpers
import numpy as np
import random
import os

# Import Stable Baseline Stuff
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

C:\Users\dilan\anaconda3\envs\deep-learning\lib\site-packages\gym\envs\registration.py:423: UserWarning: WARN: Custom namespace `ALE` is being overridden by namespace `ALE`. If you are developing a plugin you shouldn't specify a namespace in `register` calls. The namespace is specified through the entry point package metadata.
  logger.warn(
C:\Users\dilan\anaconda3\envs\deep-learning\lib\site-packages\torch\utils\tensorboard\__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(


# 2. Types of Spaces

In [2]:
Discrete(3).sample()

2

In [3]:
Box(0,1, shape=(3,)).sample()

array([0.5871597, 0.7199438, 0.5130664], dtype=float32)

In [4]:
Box(0,1, shape=(3,3)).sample()

array([[0.5257338 , 0.6113273 , 0.28437534],
       [0.72723234, 0.2335427 , 0.5208344 ],
       [0.59103125, 0.15110685, 0.5836599 ]], dtype=float32)

In [5]:
Tuple((Discrete(3), Box(0,1, shape=(3,1)))).sample()

(0,
 array([[0.7337222 ],
        [0.44291204],
        [0.8232511 ]], dtype=float32))

In [6]:
Dict({'height': Discrete(2), 'speed': Box(0,100,shape=(1,))}).sample()

OrderedDict([('height', 1), ('speed', array([57.315365], dtype=float32))])

In [7]:
MultiBinary(4).sample()

array([1, 1, 0, 1], dtype=int8)

In [8]:
MultiDiscrete([5,2,2,5]).sample()

array([2, 1, 1, 2], dtype=int64)

# 3. Building an Environment
- Build an agent to give us the best shower possible
- Randomly temperature changes between 37 to 39

In [9]:
class ShowerEnv(Env):
    def __init__(self):
        self.action_space = Discrete(3)
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        self.state = 38 + random.randint(-3,3)
        self.shower_length = 60
        
    def step(self, action):
        # Apply temp adj
        self.state += action -1
        
        # Decrease shower time
        self.shower_length -= 1
        
        # Calculate Reward
        if self.state >= 37 and self.state <= 39:
            reward = 1
        else:
            reward = -1
            
        if self.shower_length <= 0:
            done = True
        else:
            done = False
            
        info = {}
        
        return self.state, reward, done, info
    
    def render(self):
        # Implement viz
        pass
    def reset(self):
        self.state = np.array([38+random.randint(-3,3)]).astype(float)
        self.shower_length = 60
        return self.state
    

In [10]:
env = ShowerEnv()

C:\Users\dilan\anaconda3\envs\deep-learning\lib\site-packages\gym\spaces\box.py:112: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [11]:
env.observation_space.sample()

array([7.9245872], dtype=float32)

In [12]:
env.action_space.sample()

1

In [13]:
env.state

35

In [14]:
env.reset()

array([37.])

In [32]:
from stable_baselines3.common.env_checker import check_env

In [33]:
check_env(env, warn=True)

AssertionError: The observation returned by the `reset()` method does not match the given observation space

# 4. Test Environment

In [16]:
episodes = 5
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:-60
Episode:2 Score:-60
Episode:3 Score:-36
Episode:4 Score:-48
Episode:5 Score:2


# 5. Train Model

In [17]:
log_path = os.path.join('Training', 'Logs')
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [20]:
model.learn(total_timesteps=4000)

Logging to Training\Logs\PPO_17
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -25.1    |
| time/              |          |
|    fps             | 601      |
|    iterations      | 1        |
|    time_elapsed    | 3        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -25.5       |
| time/                   |             |
|    fps                  | 424         |
|    iterations           | 2           |
|    time_elapsed         | 9           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.002708342 |
|    clip_fraction        | 0.0235      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.05       |
|    explained_variance   | -0.000587   

# 6. Save Model

In [21]:
shower_path = os.path.join('Training', 'Saved Models', 'Shower_Model_PPO')

In [22]:
model.save(shower_path)

In [23]:
del model

In [24]:
model = PPO.load(shower_path, env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [25]:
model

In [27]:
evaluate_policy(model, env, n_eval_episodes=10)

(24.0, 54.99090833947008)